In [1]:
import contextily as ctx
from colorcet import palette
import datashader as ds
from datashader.mpl_ext import dsshow
import geopandas as gpd
from geopandas.tools import sjoin
import matplotlib.pyplot as plt
import pandas as pd
from pygeos import Geometry
import numpy as np
from shapely.geometry import Polygon, MultiPolygon

ModuleNotFoundError: No module named 'contextily'

In [2]:
# conda install -c conda-forge geopandas

In [6]:
# SELECT Inf.Symbol,Lin.Lname,Lin.Country,Lin.point,Sin.NewsId,COUNT(Lin.Lname)
# FROM Information as Inf
# INNER JOIN Share_in_News as Sin on Inf.SymbolId = Sin.SymbolId
# INNER JOIN Location_in_News as Lin on Sin.NewsId = Lin.NewsId 
# WHERE Inf.Symbol = 'ADVANC.BK'
# GROUP BY Lin.Lname;

In [8]:
import sqlite3
import pandas as pd
sql = """ SELECT Inf.Symbol,Lin.Lname,Lin.Country,Lin.Latitude,Lin.Longitude,COUNT(Lin.Lname) as size,N.Date
FROM Information as Inf
INNER JOIN Share_in_News as Sin on Inf.SymbolId = Sin.SymbolId
INNER JOIN News as N on N.NewsId = Sin.NewsId 
INNER JOIN Location_in_News as Lin on N.NewsId = Lin.NewsId 
WHERE Inf.Symbol = 'EE.BK'
GROUP BY Lin.Lname;
"""
con = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
cur = con.cursor()    
df = pd.read_sql(sql, con)
con.commit()
con.close

<function Connection.close>

In [2]:
df

,Symbol,Lname,Country,Latitude,Longitude,size,Date
0,EE.BK,AI + IoT,Ireland,51.846762,-8.471472,2,29/11/2022
1,EE.BK,ASPS,Iran,38.001496,46.119767,2,08/06/2022
2,EE.BK,American,United States,31.951027,-102.167630,2,08/06/2022
3,EE.BK,BC,Canada,55.001251,-125.002441,2,08/06/2022
4,EE.BK,Beauty Company,Austria,48.190201,16.337749,2,19/09/2022
5,EE.BK,CBD,India,9.156225,92.821517,2,08/06/2022
6,EE.BK,CNS,Australia,-16.877789,145.751550,2,08/06/2022
7,EE.BK,COVID-19,Serbia,43.858741,19.840092,2,08/06/2022
8,EE.BK,CW,Indonesia,-6.242439,106.858684,2,29/11/2022
9,EE.BK,Cannabis,Slovenia,46.560457,15.654405,2,19/09/2022


In [44]:
# {location.latitude}, {location.longitude}
def latitude_label(row):

    n = row.split(',')

    return float(n[0][1:])

def longitude_label(row):

    n = row.split(',')

    return float(n[1][:-1])


In [6]:
df['Latitude'] = df['point'].apply (lambda row: latitude_label(row))
df['Longitude'] = df['point'].apply (lambda row: longitude_label(row))

In [9]:
import plotly.express as px
import geopandas as gpd
# import geoplot as gplt
# ADVANC.BK
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))


px.set_mapbox_access_token(gdf)
fig = px.scatter_geo(gdf,
                    lat=gdf.geometry.y,
                    lon=gdf.geometry.x,
                    hover_name="Lname")
fig.show()

In [10]:
import plotly.express as px
# df = px.data.gapminder()
# color="Country", ADVANC.BK,animation_frame="Date",
fig = px.scatter_geo(gdf,
                    lat=gdf.geometry.y,
                    lon=gdf.geometry.x,
                    hover_name="Lname",
                    size="size",
                    
                     projection="natural earth")
fig.show()


In [14]:
import sqlite3
import pandas as pd
sql = """ SELECT *
From Location_in_News
"""
con = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
cur = con.cursor()    
df = pd.read_sql(sql, con)
con.commit()
con.close

<function Connection.close>

In [19]:
df = df.sort_values(by=['Lname'])

In [20]:
data = list(df.itertuples(index=False, name=None))

In [36]:
j = 0
l = []
for i in data:

    l.append(i[0])
    j+=1

In [39]:
def filter_special_2(words):
    # Define regular expression pattern to match words
    word_filter = []

    # Find all matches of the pattern in the text
    for word in words :
        s = 0
        for ch in word:

            if isinstance(ch , str) and ch.isnumeric(): s = 1

        if s == 0: word_filter.append(word)
                
    return list(numpy.unique(word_filter))

In [40]:
print(filter_special_2(l))

['(The Independent', '-Dr', 'A', 'A.O. Smith', 'ABBV', 'ABBV', 'ABR', 'ABT', 'ACP', 'ADAS', 'ADM', 'ADR', 'AEGEAN', 'AEGEAN', 'AEON', 'AEON', 'AEON', 'AEON', 'AEON', 'AEON', 'AEON', 'AEON', 'AEON', 'AFL', 'AGCO', 'AGCO Corporation AGCO', 'AGM', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI + IoT', 'AIS', 'AIS', 'AIS', 'AIS', 'AIS', 'AIS Business', 'AIS Fiber', 'AIS Fiber', 'AIS Insurance Service', 'ALK', 'ALS', 'AMSTERDAM', 'AMSTERDAM', 'ANET', 'ANET', 'ANI', 'AOS', 'AOV', 'AP', 'APA Corporation', 'APD', 'AQR Capital Management', 'AQR Capital Management', 'ARK Investment Management', 'ARR', 'ASEAN', 'ASIC', 'ASML Holding', 'ASML Holding N.V.', 'ASML.AS', 'ASML.AS', 'ASML.AS', 'ASML.AS', 'ASP', 'ASP', 'ASPS', 'AT&T', 'AT&T', 'AT&T', 'AT&T Inc.', 'AT&T Inc.', 'AT&T Inc.', 'ATL', 'ATLANTA', 'ATLANTA', 'ATM', 'AUM', 'AUM', 'AUM of', 'AWS', 'AWS', 'AWS', 'AXA Singapore', 'AXA Singapore', 'AXP', 'AXP', 'AYX', 'AZ', 'AZN.

In [26]:

df = pd.DataFrame(l, columns=['Lname', 'Country', 'Latitude','Longitude','NewsId'])


In [27]:
df

,Lname,Country,Latitude,Longitude,NewsId
0,ABBV,Belgium,50.859575,4.368629,261
1,ABBV,Belgium,50.859575,4.368629,335
2,ABR,Italy,42.227681,13.854983,166
3,ABT,Portugal,39.463191,-8.197362,150
4,ACP,Armenia,41.096083,44.679106,222
...,...,...,...,...,...
4065,christian,United States,39.521260,-89.282978,178
4066,congress,United States,34.153848,-112.854919,355
4067,contract.in,United States,38.386398,-85.669325,56
4068,democratic,Democratic Republic of the Congo,-2.981434,23.822264,355


In [29]:
con = sqlite3.connect(r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\Project_sofedev_2\Manage_Share\Database_convert\share_V3.sqlite')
cur = con.cursor()    
newshare = df.to_sql('Location_in_News',con,if_exists='append', index=False)
con.commit()
con.close

<function Connection.close>